## Initialization

In [1]:
guy_folder = "/content/"
cache_dir = guy_folder+"/cache/transformer_cache"

In [4]:
# %pip install matplotlib seaborn pandas tqdm tensorboard

%cd {guy_folder}
!mkdir cache
!mkdir cache/transformer_cache
# %pip install --no-cache-dir --upgrade torch torchvision
#==1.4.0+cu100 torchvision== -f https://download.pytorch.org/whl/torch_stable.html


/content
mkdir: cannot create directory ‘cache’: File exists
mkdir: cannot create directory ‘cache/transformer_cache’: File exists


In [6]:
%cd {guy_folder}/cache/
!git clone https://github.com/huggingface/transformers.git
%pip install ./transformers
%pip install -U nlp

%cd {guy_folder}

## Not working!!!
!setenv TRANSFORMERS_CACHE /vol/scratch/guy/cache/transformer_cache
!setenv CUDA_VISIBLE_DEVICES 0

/content/cache
Cloning into 'transformers'...
remote: Enumerating objects: 43860, done.
remote: Total 43860 (delta 0), reused 0 (delta 0), pack-reused 43860
Receiving objects: 100% (43860/43860), 31.63 MiB | 11.80 MiB/s, done.
Resolving deltas: 100% (30420/30420), done.
Processing ./transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.1MB 2.7MB/s 
     |████████████████████████████████| 3.0MB 16.9MB/s 
     |████████████████████████████████| 890kB 28.8MB/s 
  Created wheel for transformers: filename=transformers-3.2.0-cp36-none-any.whl size=1058986 sha256=26761964f9f482e31dcb1caed14cf0a5cd2d3accc925a1a4c3ae3dce42786abe
  Stored in directory: /tmp/pip-ephem-wheel-cache-yfsph9iq/wheels/02/9b/9d/6253b37fb4f7d72b5eeaacbbc3116710b11f600803e913a13f
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=89325

/content
/bin/bash: setenv: command not found
/bin/bash: setenv: command not found


In [2]:
%%script false
# After login in through firefox to ImageNet
%cd {guy_folder}/data
!wget http://www.image-net.org/archive/stanford/fall11_whole.tar
# !tar -xvf 

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision.datasets import ImageNet, ImageFolder, CIFAR10, CIFAR100
from torchvision import transforms
from transformers import AutoModel, AutoTokenizer

In [7]:

class DummyLayer(nn.Module):
  def __init__(self, *args, **kwargs):
    super().__init__()
    pass
  def forward(self, x, *args, **kwargs):
    return x

class PlainBERT(nn.Module):
    def __init__(self, n_tokens, min_layer = None):
        super().__init__()
        self.nLayers = 6
        self.nHeads = 12
        self.seqLen = 512


        bert = AutoModel.from_pretrained('distilbert-base-uncased', cache_dir = cache_dir)
        self.position_embeddings = nn.Parameter(torch.Tensor(bert.embeddings.position_embeddings(torch.arange(self.seqLen)).detach().numpy()))
        if min_layer is None:
          self.bert = bert.transformer
        else:
          raise NotImplementedError
          bert_ = bert.transformer
          for n, m in bert_.layer.named_children():
            if int(n) < min_layer:
              setattr(bert_.layer, n, DummyLayer())
        
          self.bert = bert_

        self.bert.requires_grad_(False)


    def forward(self, x):
        return self.bert.forward(x + self.position_embeddings, attn_mask = torch.ones(x.size(0), 512).to(x.device),
                                head_mask = torch.ones(self.nLayers, x.size(0), self.nHeads, self.seqLen, self.seqLen).to(x.device))

class BertVision(nn.Module):
    def __init__(self,  n_classes, img_dim):
        super().__init__()
        self.with_classifier = True
        self.n_tokens = np.prod(img_dim)
        self.top = nn.Sequential(
                                 nn.Conv2d(3, 32, 3, padding = 1 ),
                                 nn.LeakyReLU(0.2),
                                 nn.Conv2d(32, 100, 3, padding = 1),
                                 nn.LeakyReLU(0.2),
                                 nn.Conv2d(100, 200, 3, padding = 1),
                                 nn.LeakyReLU(0.2),
                                 nn.Conv2d(200, 768, 3, stride = (1, 2), padding = 1),
                                 nn.LeakyReLU(0.2)
                                )
        
        self.top.apply(self._init_top)
        self.bert = PlainBERT(n_tokens = self.n_tokens)
        self.fc = nn.Linear(768 * self.n_tokens//2, n_classes)
        self.layer_norm = nn.LayerNorm((512,))

    def toggleIntermediate(self):
        self.with_classifier = !self.with_classifier
    
    
    def _init_top(self, m):
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            nn.init.kaiming_uniform_(m.weight)
            m.bias.data.fill_(0.01)
        pass
    def forward(self, x):
        x = self.top(x)
        x = x.view(x.size(0), x.size(1), -1)
        x = x.transpose(1,2)
        x = self.bert(x)
        x = torch.stack(x).squeeze(0)
        x = x.transpose(1,2).contiguous()
#         x = self.layer_norm(x)
#         x = torch.mean(x, dim = (-2,))
        x = x.view(x.size(0), -1)
        if self.with_classifier:
            x = self.fc(x)
        return x.squeeze(1)

In [9]:
train_folder = "{}/data/imagenet12/train".format(guy_folder)

train_ds = ImageFolder(train_folder,
            transform = transforms.Compose([transforms.ToTensor()]))

FileNotFoundError: ignored

In [63]:
train_ds = CIFAR100("{}/data/cifar100".format(guy_folder), download = True, transform=transforms.ToTensor())

Files already downloaded and verified


In [68]:
test_ds = CIFAR100("{}/data/cifar100".format(guy_folder), download = True, transform=transforms.ToTensor(), train = False)

Files already downloaded and verified


In [14]:
device = 'cuda'
model = BertVision(len(train_ds.classes), (32,32)).to(device)

In [76]:
from collections import defaultdict
batch_size = 8
total = int(len(train_ds)/batch_size )

from tqdm import tqdm_notebook, tqdm
class_rep = defaultdict(int)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-6)
train_dataloader = DataLoader(train_ds, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(test_ds, batch_size = batch_size, shuffle = True)
pbar = tqdm(enumerate(train_dataloader), total = total, leave = True, position = 0)
acc_sum = 0

for i, (x,y) in pbar:
    if i >= total:
        break
    model.eval()
    x_test, y_test = next(iter(test_dataloader))
    test_acc = ((model(x_test.to(device)).argmax(dim =  -1) == y_test.to(device)).sum()).item()/batch_size
    model.train()
    
    optimizer.zero_grad()
    y = y.to(device)
    yhat = model(x.to(device))
    loss = criterion(yhat, y)
    acc_sum += (yhat.argmax(dim =  -1) == y).sum()
    class_rep[y[0].item()] += 1 # Incomplete
    topk = np.where(np.argsort(yhat[0].cpu().detach().numpy()) == y[0].detach().cpu().numpy())[0][0] # TODO: Incomplete
    
    pbar.set_postfix_str("Loss: {:.2f} Test Acc: {:.2f} Acc: {:.2f} Top: {} Class: {}".format(loss.item(),
                                                                                               test_acc, 
                                                                             acc_sum.item()/float(batch_size * (i+1)), 
                                                                             topk, class_rep[y[0].item()]))
    loss.backward()
    optimizer.step()

  2%|▏         | 130/6250 [01:40<1:18:23,  1.30it/s, Loss: 3.45 Test Acc: 0.12 Acc: 0.11 Top: 89 Class: 2]

KeyboardInterrupt: ignored

In [50]:
from torchvision.models import resnet101

In [64]:
vision_model = resnet101(pretrained = True)
vision_model.fc = nn.Linear(vision_model.fc.in_features, 100)
vision_model.to(device);

In [66]:
from collections import defaultdict
batch_size = 32
total = int(len(train_ds)/batch_size )

from tqdm import tqdm_notebook, tqdm
class_rep = defaultdict(int)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vision_model.parameters(), lr = 1e-2)
train_dataloader = DataLoader(train_ds, batch_size = batch_size, shuffle = True)
pbar = tqdm(enumerate(train_dataloader), total = total, leave = True, position = 0)
acc_sum = 0

for i, (x,y) in pbar:
    if i >= total:
        break
    optimizer.zero_grad()
    y = y.to(device)
    # x = transforms.Normalize(mean=[0.485, 0.456, 0.406],
    #         std=[0.229, 0.224, 0.225],)(x)
    yhat = vision_model(x.to(device))
    loss = criterion(yhat, y)
    acc_sum += (yhat.argmax(dim =  -1) == y).sum()
    class_rep[y[0].item()] += 1 # Incomplete
    topk = np.where(np.argsort(yhat[0].cpu().detach().numpy()) == y[0].detach().cpu().numpy())[0][0] # TODO: Incomplete
    pbar.set_postfix_str("Loss: {:.2f} Acc: {:.2f} Top: {} Class: {}".format(loss.item(), 
                                                                            acc_sum.item()/float(batch_size * (i+1)), 
                                                                            topk, class_rep[y[0].item()]))
    loss.backward()
    optimizer.step()

100%|█████████▉| 1561/1562 [02:02<00:00, 12.76it/s, Loss: 3.81 Acc: 0.07 Top: 93 Class: 12]

In [ ]:
y  = []
z = []
for x, _  in tqdm(train_ds):
    z.append(x)
    y.append((model(x.to(device).unsqueeze(0))).detach())

In [ ]:
y = torch.stack(y)
z = torch.stack(z)

In [ ]:
z.std()/y.std()

tensor(0.1882, device='cuda:0')

In [ ]:
i = np.random.choice(len(train_ds))
print(F.softmax(model(train_ds[i][0].unsqueeze(0).to(device))))
print(model(train_ds[i][0].unsqueeze(0).to(device)).argmax())
print(train_ds[i][1])
t = transforms.Compose([
#     transforms.Grayscale(),
    transforms.ToPILImage(),
    transforms.Grayscale(3),
    transforms.ToTensor(),
    
    ])(train_ds[i][0])
print(F.softmax(model(t.unsqueeze(0).to(device))))
print(model(t.unsqueeze(0).to(device)).argmax())

print(F.softmax(model(((train_ds[i+1][0])).unsqueeze(0).to(device))))
print(F.softmax(model(((train_ds[i+1][0])).unsqueeze(0).to(device))).argmax())


tensor([[0.0692, 0.0424, 0.1471, 0.1713, 0.0508, 0.0646, 0.0176, 0.1053, 0.2545,
         0.0771]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor(8, device='cuda:0')
5
tensor([[0.0369, 0.0232, 0.1556, 0.2595, 0.0591, 0.1124, 0.0279, 0.1635, 0.1121,
         0.0498]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor(3, device='cuda:0')
tensor([[0.0253, 0.0126, 0.1966, 0.1578, 0.0755, 0.1262, 0.0523, 0.2717, 0.0489,
         0.0330]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor(7, device='cuda:0')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


In [ ]:
model.load_state_dict(state_dict,strict = False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['fc.weight', 'fc.bias'])

In [ ]:
model.toggleIntermediate()
i = np.random.choice(len(train_ds))
a1 = (F.softmax(model(train_ds[i][0].unsqueeze(0).to(device))))

t = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Grayscale(3),
#     transforms.ColorJitter(10,10, 10),
#     transforms.RandomRotation(90),
    transforms.ToTensor(),
    
    ])(train_ds[i][0])
a2 = (F.softmax(model(t.unsqueeze(0).to(device))))

a3 = (F.softmax(model(((train_ds[i+1][0])).to(device).unsqueeze(0))))


print(torch.norm(a1-a2, p = 2))
print(torch.norm(a1-a3, p = 2))
print(torch.norm(a2-a3, p = 2))
model.toggleIntermediate()

tensor(0.1502, device='cuda:0', grad_fn=<NormBackward0>)
tensor(0.3791, device='cuda:0', grad_fn=<NormBackward0>)
tensor(0.3801, device='cuda:0', grad_fn=<NormBackward0>)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app
